## Building a Chatbot

In this video , we'll go over an example of how to design and implement an LLM-powered chatbot. this chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:
* Conversational RAG : Enable a chatbot experience over an external source of data
* Agents: Build a chatbot that can take actions


This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() # load all the environment variables

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7640ef509e40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7640ef50a4a0>, model_name='gemma2-9b-it')

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Prasanth and I am a Chief AI Engineer")])

AIMessage(content="Hello Prasanth! \n\nIt's great to meet you. That's a fascinating title! As a Chief AI Engineer, I imagine you're at the forefront of some exciting developments. \n\nWhat kind of AI projects are you currently working on?  I'm always eager to learn more about the innovative work being done in the field.\n", response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 23, 'total_tokens': 98, 'completion_time': 0.136363636, 'prompt_time': 0.001335579, 'queue_time': 0.2490547, 'total_time': 0.137699215}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e9d989d-476c-4a63-85c8-dd24ac5628eb-0', usage_metadata={'input_tokens': 23, 'output_tokens': 75, 'total_tokens': 98})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Prasanth and I am a Chief AI Engineer"),
        AIMessage(content="Hello Prasanth,\n\nIt's nice to meet you!  \n\nAs a Chief AI Engineer, that's a fascinating role. What kind of projects are you working on these days?  \n\nI'm always eager to learn more about the cutting edge of AI.\n"),
        HumanMessage(content="Hey, what's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Prasanth and that you are a Chief AI Engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! \n", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 103, 'total_tokens': 150, 'completion_time': 0.085454545, 'prompt_time': 0.002767598, 'queue_time': 0.257137891, 'total_time': 0.088222143}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-82f7a290-54d6-4a97-8212-47e80dd3f5bf-0', usage_metadata={'input_tokens': 103, 'output_tokens': 47, 'total_tokens': 150})

## Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs 
and outputs of the model, and store them in some datastore, future interactions will then load those messsages and pass them into the chain as part of the input. Let's see how to use this!



In [ ]:
# Import chat message history class (in-memory store of messages)
from langchain_community.chat_message_histories import ChatMessageHistory
# Base class for chat histories (helps enforce type consistency)
from langchain_core.chat_history import BaseChatMessageHistory
# Wrapper that lets a Runnable (like an LLM) work with conversation memory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dictionary to store chat histories for multiple sessions
# Keys   -> session_id (string)
# Values -> ChatMessageHistory object (conversation memory for that session)
store = {}


# Function to fetch (or create) the chat history for a given session
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        # If this session is new, create a fresh history object
        # ChatMessageHistory stores HumanMessage/AIMessage automatically
        store[session_id] = ChatMessageHistory()
    
    # Return the history for this session
    return store[session_id]


# Wrap the model with message history support
# - model: your LLM (Runnable, e.g., OpenAI, Groq, etc.)
# - get_session_history: function that retrieves the right history for each session
with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [16]:
config = {"configurable" : {"session_id" : "chat1"}}

In [17]:
with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Prasanth and I am a Chief AI Engineer")],
    config = config
)

AIMessage(content="Hello Prasanth! It's a pleasure to meet you. That's a fascinating title - Chief AI Engineer. What kind of work do you do in that role?  \n\nI'm eager to learn more about your experience and expertise in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 23, 'total_tokens': 81, 'completion_time': 0.105454545, 'prompt_time': 0.001334719, 'queue_time': 0.254493871, 'total_time': 0.106789264}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-40e39b56-81a1-427e-964a-6ed84983f21a-0', usage_metadata={'input_tokens': 23, 'output_tokens': 58, 'total_tokens': 81})

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Prasanth.  You told me at the beginning of our conversation! 😊 \n\n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 95, 'total_tokens': 128, 'completion_time': 0.06, 'prompt_time': 0.002604118, 'queue_time': 0.254511362, 'total_time': 0.062604118}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-84b5f74c-0f97-4d97-b0b9-e4527711e7e4-0', usage_metadata={'input_tokens': 95, 'output_tokens': 33, 'total_tokens': 128})

In [19]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and no way of knowing your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Hi John!  Nice to meet you. 👋 \n\nIs there anything I can help you with today?\n'

In [21]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John! 😊  \n\nI remember that you told me earlier.  \n\n\n\n'

### Prompt Template


Prompt Templates help to turn raw user information into a foramt that the LLM can work with. In this case, the raw user input is just a message, which are passing to the LLM. Let's now make that a bit more complicated. First let's add a system message with some custom instructions (but still taking messages as input). Next, we'll add more input besides just the messages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpfule assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [25]:
# As we already create messages as a variable name as placeholder, here we need to pass messsages as a key
chain.invoke({"messages":  [HumanMessage(content="Hi, My name is Prasanth")]})

AIMessage(content="Hello Prasanth, it's nice to meet you! \n\nI'm happy to help with any questions you have.  What can I do for you today?  \n\n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 34, 'total_tokens': 74, 'completion_time': 0.072727273, 'prompt_time': 0.00148219, 'queue_time': 0.252544609, 'total_time': 0.074209463}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e7b4602-7ac9-4c1d-a3aa-1ac3ce4a81b7-0', usage_metadata={'input_tokens': 34, 'output_tokens': 40, 'total_tokens': 74})

In [26]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [30]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is prasanth")],
    config=config
)

response

AIMessage(content="Hi Prasanth! \n\nIt's great to meet you too.  What can I do for you today? 😄  \n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 72, 'total_tokens': 101, 'completion_time': 0.052727273, 'prompt_time': 0.002293939, 'queue_time': 0.25863482, 'total_time': 0.055021212}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ced17e1f-8cf4-4c7e-9422-44e33486cfee-0', usage_metadata={'input_tokens': 72, 'output_tokens': 29, 'total_tokens': 101})

In [32]:
config={"configurable":{"session_id":"chat3"}}

response = with_message_history.invoke(
    [
        HumanMessage(content="what's my name")
    ],
    config = config
)

response.content

"Your name is Prasanth! 😊 \n\n\nLet me know if you have any other questions. I'm here to help!\n"

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [37]:
response = chain.invoke({"messages": [HumanMessage(content="Hi My name is prasanth")], "language": "telugu"})
response.content

'నమస్తే ప్రశాంత్! 😊  \n\nమీకు ఎలా సహాయం చేయగలను? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.


In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am prasanth")],"language":"Telugu"},
    config=config
)
repsonse.content

'మీకు నమస్కారం, ప్రశాంత్! 😊 \n\nనేను మీకు సహాయకరమైన సహాయకుడిని. ఏవేదో ప్రశ్నించండి, నా సోదరుడిగానే సమాధానాలు ఇస్తాను.  \n\n'

## Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. if left unmanaged, the list of messages will go unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of messages you are passing in.

`trim_messages`: helper to reduce how many messages we're sending to the model. the trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system messages and whether to allow partial messages

In [44]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)


messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [45]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what ice cream do i like?")],
        "language": "English"
    }
)

AIMessage(content='You said you like vanilla ice cream! 😊 🍦  \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 121, 'total_tokens': 136, 'completion_time': 0.027272727, 'prompt_time': 0.003295578, 'queue_time': 0.250771461, 'total_time': 0.030568305}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd1ab339-55df-40eb-9981-f824896a9164-0', usage_metadata={'input_tokens': 121, 'output_tokens': 15, 'total_tokens': 136})

In [46]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [47]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob, you told me earlier! 😊  \n'